In [1]:
import os
import json
import obspy
import numpy as np
import time

def evtDataToJSON(base_path):
    os.chdir(base_path)
    earthquake_list = {}
    for dir in os.listdir("."):
        if os.path.isdir(dir):
            os.chdir(dir)
            for file_name in os.listdir("."):
                if file_name.endswith(".txt"):
                    with open(file_name, "r", encoding="shift_jis") as file:
                        evt_data = {}
                        for line in file:
                            key, value = map(str.strip, line.split(":", 1))
                            evt_data[key] = value
                        earthquake_list[file_name] = evt_data

            os.chdir(base_path) 

    output_file = "earthquake_data.json"
    with open(output_file, "w", encoding="utf-8") as json_file:
        json.dump(earthquake_list, json_file, ensure_ascii=False, indent=4)

    print(f"Earthquake data saved to {output_file}")
    return output_file

def evtResultantVectorByStation(event_dir_path):
    files_only = [f for f in os.listdir(event_dir_path) if os.path.isfile(os.path.join(event_dir_path, f))]
    magnitude_by_station = {}
    for f in files_only:
        if f.endswith(".SAC"):
            current_file = obspy.read(os.path.join(event_dir_path, f), debug_headers=True)
            station = f.split(".")[-3]

            # trace for a single direction of a single station for this event
            for trace in current_file:
                if station not in magnitude_by_station:
                    magnitude_by_station[station] = np.zeros(len(trace.data))
                for i, v in enumerate(trace.data):
                    magnitude_by_station[station][i] += v**2
    for key in magnitude_by_station:
        magnitude_by_station[key] = np.sqrt(magnitude_by_station[key])
    return magnitude_by_station

def evtArray(base_path):
    num_events = sum(1 for entry in os.scandir(base_path) if entry.is_dir())
    print(f"Processing a total of {num_events} events.")
    events = []
    processed_events = 0
    for dir in os.listdir(base_path):
        full_path = os.path.join(base_path, dir)
        if os.path.isdir(full_path):
            print(f"Processing event {dir}...")
            start_time = time.perf_counter()
            events.append(evtResultantVectorByStation(full_path))
            end_time = time.perf_counter()
            iteration_time = end_time - start_time
            processed_events += 1
            print(f"Event {dir} processed sucessfully in {iteration_time:.6f} seconds! [{processed_events}/{num_events}]")
    return events

if __name__ == "__main__":
    # metadata
    base_path = "/home/gjnagatomo/seismon-jp/data/event_data/2025-01-15_14-22-35/"
    json_file = evtDataToJSON(base_path)
    os.chdir(base_path)

    with open(json_file, "r") as file:
        data = json.load(file)
    
    events = evtArray(base_path)

    cnt_base_path = "/home/gjnagatomo/seismon-jp/data/continuous_data/2025-01-21_18-22-00"
    continuous = evtArray(cnt_base_path)

    # TODO: Change evt functions' names cause they can be used for cnt data 


/usr/local/lib/python3.6/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


Earthquake data saved to earthquake_data.json
Processing a total of 25 events.
Processing event D20120101000394_20...
Event D20120101000394_20 processed sucessfully in 22.866890 seconds! [1/25]
Processing event D20120101000501_20...
Event D20120101000501_20 processed sucessfully in 8.529054 seconds! [2/25]
Processing event D20120103000097_20...
Event D20120103000097_20 processed sucessfully in 13.410215 seconds! [3/25]
Processing event D20120106000056_20...
Event D20120106000056_20 processed sucessfully in 21.448006 seconds! [4/25]
Processing event D20120109000186_20...
Event D20120109000186_20 processed sucessfully in 21.360378 seconds! [5/25]
Processing event D20120110000529_20...
Event D20120110000529_20 processed sucessfully in 2.950625 seconds! [6/25]
Processing event D20120112000250_20...
Event D20120112000250_20 processed sucessfully in 63.602778 seconds! [7/25]
Processing event D20120112000325_20...
Event D20120112000325_20 processed sucessfully in 39.136491 seconds! [8/25]
Pro